<a href="https://colab.research.google.com/github/kgautam2103/EventBrite_Analytics/blob/main/get_attendee_details_Org_list_gsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime,timedelta,date
from google.colab import files



def attendee_details_list_fn(attendee_response_list,org_id,org_name):
  attendee_details_list=[]
  for attendee_detail in attendee_response_list:
    answer_1 = ""
    answer_2 = ""
    answer_3 = ""
    attendee_id = attendee_detail['id']
    created_datetime = attendee_detail['created']
    event_id = attendee_detail['event_id']
    order_id = attendee_detail['order_id']
    ticket_name = attendee_detail['ticket_class_name']
    attendee_name = attendee_detail['profile']['name']
    attendee_email = attendee_detail['profile']['email']
    created_date = datetime.strptime(created_datetime[:-10],'%Y-%m-%d').date()
    status = attendee_detail['status']
    attendee_answers = attendee_detail['answers']
    try:
      answer_1 = attendee_answers[0]['answer']
      answer_2 = attendee_answers[1]['answer']
      answer_3 = attendee_answers[2]['answer']
    except:
      pass

    attendee_details_list.append([org_id,org_name,attendee_id,order_id,event_id,attendee_name, attendee_email,created_date,created_datetime,status,answer_1,answer_2,answer_3])
  return attendee_details_list

def event_details_list_fn(order_details_df,countrycode_df):
  order_event_details_list=[]
  for index,row in order_details_df.iterrows():
    org_id = row['org_id']
    org_name = row['org_name']
    attendee_id = row['attendee_id']
    order_id = row['order_id']
    event_id = row['event_id']
    attendee_name = row['attendee_name']
    attendee_email = row['attendee_email']
    created_datetime = str(row['created_datetime'])
    created_date = str(row['created_date'])
    status = row['status']
    answer_1 = row['answer_1']
    answer_2 = row['answer_2']
    answer_3 = row['answer_3']
    get_event_url = prod_url+events+event_id
    get_event_response = requests.get(get_event_url,headers=auth_headers)


    event_name = get_event_response.json()['name']['text']
    start_date = get_event_response.json()['start']['local']
    timezone_value = get_event_response.json()['start']['timezone']
    locale = get_event_response.json()['locale']

    try:
      venue_id = get_event_response.json()['venue_id']
      get_venue_url = prod_url+venues+venue_id
      get_venue_response = requests.get(get_venue_url,headers=auth_headers)
      city_name = get_venue_response.json()['address']['city']
      #print(city_name)
      country_name = get_venue_response.json()['address']['country']
      #print(country_name)
      country_df = countrycode_df[countrycode_df.alpha2==country_name]['country']
      code_df = countrycode_df[countrycode_df.alpha2==country_name]['country_code']
      country=country_df.to_list()[0]
      #print()
      countrycode = code_df.to_list()[0]
    except:
      city_name=""
      country=""
      countrycode=''
    #order_details_list.append([order_id,event_id,name,email,timezone_value,start_date,event_name])
    order_event_details_list.append([org_id,org_name,attendee_id,order_id,event_id,attendee_name, attendee_email,created_date,created_datetime,status,answer_1,answer_2,answer_3,locale,event_name, start_date, timezone_value,city_name,country,countrycode])
  return order_event_details_list



In [ ]:
#define global variables
auth_token = 'J5XYRMHHONHEQUPII5SG'
prod_url = 'https://www.eventbriteapi.com/v3/'
org = '/organizations/'
orders = '/orders/'
attendees = '/attendees/'
events='/events/'
venues = '/venues/'


authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1XXQkO8HX-DnRLlgGF_zX1y6CZmoFcEBRnHCgDNIITC8/edit#gid=0')

sheet_name = input('enter the name of input sheet to process: ')
input_sheet = wb.worksheet(sheet_name)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['org_id','org_name','is_date_filter',	'start_date',	'end_date'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

output_sheet_url = input('enter the url of output sheet to process: ')
output_wb = gc.open_by_url(output_sheet_url)
output_sheet_name = input('enter the name of output sheet use start and end date as name like 01012023-to-01312023: ')
#output_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1co21FptC1MZztR5QgrTheLWTriroQ2dCYOrNFKq-Yro/edit#gid=0')
try:
  combined_sheet = output_wb.worksheet("Combined")
except:
  print("sheet does not exist")
  combined_sheet = output_wb.add_worksheet(title="Combined", rows="10000", cols="30")

try:
  event_worksheet = output_wb.worksheet(output_sheet_name)
except:
  print("sheet does not exist")
  event_worksheet = output_wb.add_worksheet(title=output_sheet_name, rows="2000", cols="30")

combined_df = pd.DataFrame(combined_sheet.get_all_records())
event_final_df = pd.DataFrame(event_worksheet.get_all_records())

countrycode_wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1M20GbXcIZCGwWfCdq8NWeCs3Xc_g4CWdBsjxRtuTeM0/edit#gid=0")
lookup_sheet = countrycode_wb.worksheet("lookup")
Country_code = lookup_sheet.get_all_values()
countrycode_df = pd.DataFrame(Country_code, columns=['country','country_code','alpha2','alpha3'])
countrycode_df = countrycode_df.drop(countrycode_df.index[0])

for index,row in inputdata_df.iterrows():
  print(str(row['org_id']),str(row['org_name']))
  attendee_details_list=[]
  org_id = str(row['org_id'])
  org_name = str(row['org_name'])
  is_date_filter = str(row['is_date_filter']).lower()
  start_date = str(row['start_date'])
  end_date = str(row['end_date'])

  if is_date_filter == 'true':
    filter_start_date = datetime.strptime(start_date,'%Y-%m-%d').date()
    filter_end_date = datetime.strptime(end_date,'%Y-%m-%d').date()

  attendee_details_url = prod_url+org+org_id+attendees

  get_attendee_details_response = requests.get(attendee_details_url,headers=auth_headers)

  print(get_attendee_details_response.status_code)

  if get_attendee_details_response.status_code == 200:
    has_more_items = get_attendee_details_response.json()['pagination']['has_more_items']
    if has_more_items:
        continuation = get_attendee_details_response.json()['pagination']['continuation']
    attendee_response_list = get_attendee_details_response.json()['attendees']
    attendee_details_list.extend(attendee_details_list_fn(attendee_response_list,org_id,org_name))



    while has_more_items:
      paginated_attendee_details_url = prod_url+org+org_id+attendees+"?continuation="+continuation
      paginated_get_attendee_details_response = requests.get(paginated_attendee_details_url,headers=auth_headers)
      has_more_items = paginated_get_attendee_details_response.json()['pagination']['has_more_items']
      if has_more_items:
        continuation = paginated_get_attendee_details_response.json()['pagination']['continuation']
      paginated_attendee_response_list = paginated_get_attendee_details_response.json()['attendees']
      len(paginated_attendee_response_list)
      attendee_details_list.extend(attendee_details_list_fn(paginated_attendee_response_list,org_id,org_name))


  attendee_details_df = pd.DataFrame(attendee_details_list,columns=['org_id','org_name','attendee_id','order_id','event_id','attendee_name', 'attendee_email','created_date','created_datetime','status','answer_1','answer_2','answer_3'])

  if is_date_filter == 'true':
    attendee_details_df = attendee_details_df[attendee_details_df.created_date>=filter_start_date]
    attendee_details_df = attendee_details_df[attendee_details_df.created_date<=filter_end_date]

  attendee_event_details_list = event_details_list_fn(attendee_details_df,countrycode_df)
  attendee_event_details_df = pd.DataFrame(attendee_event_details_list,columns=['org_id','org_name','attendee_id','order_id','event_id','attendee_name', 'attendee_email','created_date','created_datetime','status','answer_1','answer_2','answer_3','locale','event_name','event_start_date','event_timezone','city','country','countrycode'])

  attendee_event_details_df.drop_duplicates(subset=['attendee_email'],inplace=True)

  if event_final_df.empty:
    event_final_df = attendee_event_details_df
  else:
    event_final_df = pd.concat([event_final_df, attendee_event_details_df], axis=0)

  event_final_df.drop_duplicates(subset=['attendee_email'],inplace=True)
  event_worksheet.update([event_final_df.columns.values.tolist()] + event_final_df.values.tolist())


  combined_df = pd.DataFrame(combined_sheet.get_all_records())
  if combined_df.empty:
    print("its empty")
    final_df = attendee_event_details_df
  else:
    final_df = pd.concat([combined_df, attendee_event_details_df], axis=0)
    final_df.drop_duplicates(subset=['attendee_email'],inplace=True)
  combined_sheet.update([final_df.columns.values.tolist()] + final_df.values.tolist())


enter the name of input sheet to process: historical_data
enter the url of output sheet to process: https://docs.google.com/spreadsheets/d/1PErW7vTFYgRIDgBO4C_wFrzYVCrtkea-g28sy_zeCMU/edit#gid=748866969
enter the name of output sheet use start and end date as name like 01012023-to-01312023: signups_till_Jan29
sheet does not exist
1227441422413 SY Meditation USA
200
